In [1]:
import numpy as np
from functools import partial
import pickle
import matplotlib.pyplot as plt

In [2]:
def rdm_padding(rdm, Z):
    num_orbitals = np.array([5 if z < 3 else 14 for z in Z])
    rdm_atom_start_indices = np.concatenate([[0], num_orbitals]).cumsum()
    insert_start_positions = rdm_atom_start_indices[1:][rdm_atom_start_indices[1:] - rdm_atom_start_indices[:-1] == 5]
    insertum = np.zeros((9, rdm.shape[1]))
    for index in reversed(insert_start_positions):
        rdm = np.insert(rdm, index, insertum, axis=0)
    rdm = rdm.T
    insertum = np.zeros((9, rdm.shape[1]))
    for index in reversed(insert_start_positions):
        rdm = np.insert(rdm, index, insertum, axis=0)
    for i in np.concatenate([[0], (14 * np.ones_like(Z, dtype=np.int32))[:-1]]).cumsum()[Z <= 2] + 2:
        rdm[:, [i, i + 1, i + 2, i + 3]] = rdm[:, [i + 3, i, i + 1, i + 2]]
        rdm[[i, i + 1, i + 2, i + 3], :] = rdm[[i + 3, i, i + 1, i + 2], :]
    return rdm

In [3]:
def rdm_reshape(rdm):
    side_length = len(rdm)
    n_atoms = side_length // 14
    rdm = np.reshape(rdm, (n_atoms, 14, n_atoms, 14))
    rdm = np.transpose(rdm, (0, 2, 1, 3))
    rdm = np.reshape(rdm, (n_atoms ** 2, 14, 14 ))
    return rdm

In [4]:
def add_Z_and_N(dict, Z):
    n_mols = len(dict["R"])
    N = Z.shape[0] * np.ones((n_mols,), dtype=np.int32)
    dict["Z"] = np.tile(Z, n_mols)
    dict["N"] = np.tile(N, n_mols)
    return dict

**H2**

In [5]:
# data = np.load("../../data/md_h2.npz", allow_pickle=True)
# Z = np.array([1, 1])
# data = add_Z_and_N(data, Z)
# rdm_hf = data["hf_rdm"]
# h2_rdm_pad = partial(rdm_padding, Z=Z)
# rdm_hf = np.vectorize(h2_rdm_pad, signature="(a,a)->(b,b)")(rdm_hf)
# print(rdm_hf.shape)
# rdm_hf = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_hf)
# rdm_hf = np.reshape(rdm_hf, (rdm_hf.shape[0] * rdm_hf.shape[1], rdm_hf.shape[2], rdm_hf.shape[3]))
# data["hf_rdm"] = rdm_hf
# data["N_rdm"] = data["N"] ** 2
# # restrict train coords to the useful ones
# for key in ["train_coords", "hf_train_density", "mp_train_density"]:
#     data[key] = data[key][:, :140]

In [7]:
#with open("../../data/md_h2.npz", "wb") as f:
#     pickle.dump(data, f)

**Benzene**

In [ ]:
data = np.load("../../data/md_benzene_mod.npz", allow_pickle=True)
Z = np.array([6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1])
data = add_Z_and_N(data, Z)
rdm_hf = data["hf_rdm"]
h2_rdm_pad = partial(rdm_padding, Z=Z)
rdm_hf = np.vectorize(h2_rdm_pad, signature="(a,a)->(b,b)")(rdm_hf)
rdm_hf = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_hf)
rdm_hf = np.reshape(rdm_hf, (rdm_hf.shape[0] * rdm_hf.shape[1], rdm_hf.shape[2], rdm_hf.shape[3]))
data["hf_rdm"] = rdm_hf
data["N_rdm"] = data["N"] ** 2

In [ ]:
with open("../../data/md_benzene_mod.npz", "wb") as f:
    pickle.dump(data, f)

**Formamide**

In [9]:
data = np.load("../../data/formamide.npz", allow_pickle=True)
Z = np.array([7, 1, 1, 6, 8, 1])
data = add_Z_and_N(data, Z)

# Hartree-Fock RDM
rdm_hf = data["hf_1rdms"]
rdm_pad = partial(rdm_padding, Z=Z)
rdm_hf = np.vectorize(rdm_pad, signature="(a,a)->(b,b)")(rdm_hf)
rdm_hf = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_hf)
rdm_hf = np.reshape(rdm_hf, (rdm_hf.shape[0] * rdm_hf.shape[1], rdm_hf.shape[2], rdm_hf.shape[3]))
data["hf_1rdms"] = rdm_hf

# MP2 RDM
rdm_mp = data["mp_1rdms"]
rdm_pad = partial(rdm_padding, Z=Z)
rdm_mp = np.vectorize(rdm_pad, signature="(a,a)->(b,b)")(rdm_mp)
rdm_mp = np.vectorize(rdm_reshape, signature="(a,a)->(c,b,b)")(rdm_mp)
rdm_mp = np.reshape(rdm_mp, (rdm_mp.shape[0] * rdm_mp.shape[1], rdm_mp.shape[2], rdm_mp.shape[3]))
data["mp_1rdms"] = rdm_mp

data["N_rdm"] = data["N"] ** 2

In [10]:
with open("../../data/formamide.npz", "wb") as f:
    pickle.dump(data, f)

In [11]:
data = np.load("../../data/formamide.npz", allow_pickle=True)
data.keys()

dict_keys(['R', 'grid_coordss', 'sphere_coordss', 'hf_1rdms', 'mp_1rdms', 'hf_grid_densities', 'hf_sphere_densities', 'mp_grid_densities', 'mp_sphere_densities', 'dft_grid_densities', 'dft_sphere_densities', 'Z', 'N', 'N_rdm'])

In [12]:
data["mp_1rdms"].shape

(33552, 14, 14)